In [19]:
import time
import pandas as pd
import numpy as np

def read_csv(filepath):
    '''
    TODO : This function needs to be completed.
    Read the events.csv and mortality_events.csv files. 
    Variables returned from this function are passed as input to the metric functions.
    '''
    events = pd.read_csv(filepath + 'events.csv')
    mortality = pd.read_csv(filepath + 'mortality_events.csv')
    return events, mortality

train_path = '../data/train/'
events, mortality = read_csv(train_path)
print(events)
print(mortality)

        patient_id      event_id  \
0               12    DIAG440649   
1               12    DIAG201070   
2               12    DIAG321462   
3               12    DIAG321042   
4               12    DIAG435141   
5               12    DIAG320128   
6               12    DIAG195581   
7               12    DIAG198464   
8               12    DIAG443862   
9               19    DIAG433329   
10              19     DIAG81902   
11              19     DIAG74082   
12              19    DIAG436539   
13              19     DIAG80502   
14              19    DIAG373995   
15              19    DIAG374009   
16              19    DIAG320128   
17              19    DIAG321873   
18              41    DIAG441806   
19              41    DIAG261881   
20              41    DIAG257010   
21              41    DIAG259852   
22              41  DIAG40480033   
23              41    DIAG252663   
24              41    DIAG200485   
25              41    DIAG260041   
26              41    DIAG25

In [20]:
patient_events = events.groupby('patient_id').size().reset_index(name='event_count')
dead_patient_events = patient_events.loc[
    patient_events.patient_id.isin(mortality.patient_id)
]
alive_patient_events = patient_events.loc[
    ~patient_events.patient_id.isin(mortality.patient_id)
]

print(dead_patient_events.event_count.mean())
print(dead_patient_events.event_count.max())
print(dead_patient_events.event_count.min())
print(alive_patient_events.event_count.mean())
print(alive_patient_events.event_count.max())
print(alive_patient_events.event_count.min())

982.014
8635
1
498.118
12627
1


In [21]:
patient_encounter_events = events.groupby(
    ['patient_id', 'timestamp']
).size().groupby('patient_id').size().reset_index(name='encounter_event_count')
dead_patient_encounter_events = patient_encounter_events.loc[
    patient_events.patient_id.isin(mortality.patient_id)
]
alive_patient_encounter_events = patient_encounter_events.loc[
    ~patient_events.patient_id.isin(mortality.patient_id)
]

print(dead_patient_encounter_events.encounter_event_count.mean())
print(dead_patient_encounter_events.encounter_event_count.max())
print(dead_patient_encounter_events.encounter_event_count.min())
print(alive_patient_encounter_events.encounter_event_count.mean())
print(alive_patient_encounter_events.encounter_event_count.max())
print(alive_patient_encounter_events.encounter_event_count.min())

23.038
203
1
15.452
391
1


In [22]:
events['timestamp'] = pd.to_datetime(events['timestamp'])
events = events.sort_values(
    ['patient_id', 'timestamp'], ascending=[True, True]
)

def record_length(patient_series):
    return (patient_series.max() - patient_series.min()).days
patient_record_lengths = events.groupby('patient_id')['timestamp'].agg([record_length]).reset_index(level=0)
dead_patient_record_lengths = patient_record_lengths.loc[
    patient_record_lengths.patient_id.isin(mortality.patient_id)
]
alive_patient_record_lengths = patient_record_lengths.loc[
    ~patient_record_lengths.patient_id.isin(mortality.patient_id)
]
dead_patient_record_lengths

,patient_id,record_length
0,12,15
1,19,4
2,41,43
5,106,6
6,112,839
7,149,8
8,157,38
9,177,389
10,181,0
13,224,1


In [23]:
from datetime import datetime, timedelta

is_dead_boolean_series = events.patient_id.isin(mortality.patient_id)

dead_patients_df = mortality.copy()
dead_patients_index_date_series = dead_patients_df.timestamp.map(
    lambda mortality_date: datetime.strftime(
        datetime.strptime(mortality_date, "%Y-%m-%d") - timedelta(days=30),
        "%Y-%m-%d"
    )
)

dead_patients_df = dead_patients_df.drop(['timestamp', 'label'], axis=1)
dead_patients_df['indx_date'] = dead_patients_index_date_series

dead_patients_df

,patient_id,indx_date
0,19,2014-02-02
1,12,2011-11-19
2,41,2014-01-16
3,106,2015-07-12
4,112,2011-03-24
5,149,2012-01-19
6,157,2013-02-26
7,177,2010-03-30
8,181,2015-05-04
9,224,2014-09-04


In [24]:
is_dead_boolean_series = events.patient_id.isin(mortality.patient_id)
events['timestamp'] = pd.to_datetime(events['timestamp'], format='%Y-%m-%d')
alive_patient_events_df = events.loc[
    ~is_dead_boolean_series
]

alive_patients_df = alive_patient_events_df.groupby('patient_id')['timestamp'].max().reset_index(
    name='indx_date'
)
alive_patients_df.indx_date = alive_patients_df.indx_date.dt.strftime('%Y-%m-%d')

print(alive_patients_df.dtypes)
alive_patients_df

patient_id     int64
indx_date     object
dtype: object


,patient_id,indx_date
0,80,2015-01-27
1,99,2013-10-13
2,197,2013-11-16
3,198,2010-04-18
4,306,2010-09-12
5,388,2012-06-25
6,397,2013-11-25
7,402,2012-10-23
8,421,2012-08-04
9,424,2013-07-04


In [25]:
indx_date_df = pd.concat([alive_patients_df, dead_patients_df])
indx_date_df

,patient_id,indx_date
0,80,2015-01-27
1,99,2013-10-13
2,197,2013-11-16
3,198,2010-04-18
4,306,2010-09-12
5,388,2012-06-25
6,397,2013-11-25
7,402,2012-10-23
8,421,2012-08-04
9,424,2013-07-04


In [26]:
indx_date_df.to_csv(
    '../data/train/etl_index_dates.csv', 
    columns=['patient_id', 'indx_date'], 
    index=False
)

In [27]:
joined_df = events.join(indx_date_df.set_index('patient_id'), on='patient_id')
joined_df.indx_date = pd.to_datetime(joined_df.indx_date, format='%Y-%m-%d')

joined_df = joined_df[
    (joined_df.indx_date >= joined_df.timestamp) & 
    (joined_df.indx_date - joined_df.timestamp <= timedelta(days=2000))
]
joined_df = joined_df.drop(['event_description', 'timestamp'], axis=1)
joined_df

,patient_id,event_id,value,indx_date
9,19,DIAG433329,1.00,2014-02-02
10,19,DIAG81902,1.00,2014-02-02
11,19,DIAG74082,1.00,2014-02-02
12,19,DIAG436539,1.00,2014-02-02
13,19,DIAG80502,1.00,2014-02-02
14,19,DIAG373995,1.00,2014-02-02
15,19,DIAG374009,1.00,2014-02-02
16,19,DIAG320128,1.00,2014-02-02
17,19,DIAG321873,1.00,2014-02-02
218905,19,LAB3048529,0.04,2014-02-02


In [28]:
joined_df.to_csv(
    '../data/train/etl_filtered_events.csv', 
    columns=['patient_id', 'event_id', 'value'], 
    index=False
)

In [38]:
filtered_events_df = joined_df.copy()
feature_map_df = pd.read_csv('../data/train/event_feature_map.csv')

# Join feature_map_df and filtered_events_df using event_id, using 
# corresponding idx of feature_map_df as the new feature_id
aggregated_events = filtered_events_df.join(
    feature_map_df.set_index('event_id'), 
    on='event_id'
)
aggregated_events = aggregated_events.rename({'idx': 'feature_id'}, axis='columns')

# Remove events with NaN values
aggregated_events = aggregated_events[pd.notnull(aggregated_events.value)]

# Aggregate events with sum and count
aggregated_events = aggregated_events.groupby(
    ['patient_id', 'event_id', 'feature_id']
)['value'].agg(['sum', 'count']).reset_index()
aggregated_events['feature_value'] = aggregated_events.apply(
    lambda x: 
    x['sum'] 
    if x.event_id.startswith('DIAG') or x.event_id.startswith('DRUG') 
    else x['count'],
    axis=1
)

# Normalize feature_value
max_feature_value_df = aggregated_events.groupby(
    ['feature_id']
)['feature_value'].max().reset_index(
    name='max_feature_value'
)
aggregated_events = aggregated_events.join(
    max_feature_value_df.set_index('feature_id'), 
    on='feature_id'
)
aggregated_events['feature_value'] = (
    aggregated_events.feature_value / aggregated_events.max_feature_value
)

# Remove unneeded columns
aggregated_events = aggregated_events.drop(
    ['event_id', 'sum', 'count', 'max_feature_value'],
    axis=1
)
aggregated_events

,patient_id,feature_id,feature_value
0,19,719,0.250000
1,19,744,1.000000
2,19,784,0.333333
3,19,786,0.333333
4,19,1202,1.000000
5,19,1411,1.000000
6,19,1895,1.000000
7,19,2001,0.500000
8,19,2028,0.500000
9,19,2040,0.090909


In [30]:
aggregated_events.to_csv(
    '../deliverables/etl_aggregated_events.csv', 
    columns=['patient_id', 'feature_id', 'feature_value'], 
    index=False
)

In [31]:
patient_features = {}
aggregated_events = aggregated_events.copy()

# Create a 'tuplet' column of (feature_id, feature_value) for each aggregate event
aggregated_events['tuplet'] = list(zip(
    aggregated_events.feature_id, aggregated_events.feature_value
))
# Sort based on feature_id
aggregated_events = aggregated_events.sort_values(by=['feature_id'])

for aggregated_event in aggregated_events.itertuples():
    patient_feature = patient_features.get(aggregated_event.patient_id, [])
    patient_feature.append(aggregated_event.tuplet)
    patient_features[aggregated_event.patient_id] = patient_feature
patient_features

{19: [(719, 0.0018975332068311196),
  (744, 0.0018975332068311196),
  (784, 0.0018975332068311196),
  (786, 0.0018975332068311196),
  (1202, 0.0018975332068311196),
  (1411, 0.0018975332068311196),
  (1895, 0.0018975332068311196),
  (2001, 0.0018975332068311196),
  (2028, 0.0018975332068311196),
  (2040, 0.003795066413662239),
  (2045, 0.003795066413662239),
  (2075, 0.003795066413662239),
  (2221, 0.003795066413662239),
  (2260, 0.0018975332068311196),
  (2340, 0.0018975332068311196),
  (2378, 0.003795066413662239),
  (2432, 0.003795066413662239),
  (2435, 0.0018975332068311196),
  (2447, 0.0018975332068311196),
  (2448, 0.0018975332068311196),
  (2503, 0.0018975332068311196),
  (2593, 0.0018975332068311196),
  (2598, 0.003795066413662239),
  (2599, 0.003795066413662239),
  (2640, 0.003795066413662239),
  (2646, 0.0018975332068311196),
  (2682, 0.0018975332068311196),
  (2684, 0.0018975332068311196),
  (2688, 0.0018975332068311196),
  (2696, 0.007590132827324478),
  (2697, 0.001897533

In [32]:
mortality_df = mortality.copy()
all_patients = events.patient_id.unique()

mortality_dict = dict(zip(all_patients, 
         map(lambda is_dead: 0 if is_dead else 1, np.isin(all_patients, mortality_df.patient_id))
        ))
mortality_dict

{12: 0,
 19: 0,
 41: 0,
 80: 1,
 99: 1,
 106: 0,
 112: 0,
 149: 0,
 157: 0,
 177: 0,
 181: 0,
 197: 1,
 198: 1,
 224: 0,
 279: 0,
 281: 0,
 284: 0,
 294: 0,
 306: 1,
 308: 0,
 329: 0,
 388: 1,
 397: 1,
 402: 1,
 407: 0,
 421: 1,
 424: 1,
 453: 0,
 462: 1,
 465: 1,
 471: 0,
 472: 0,
 499: 0,
 512: 1,
 528: 1,
 559: 0,
 618: 0,
 634: 0,
 635: 1,
 657: 1,
 679: 0,
 701: 1,
 704: 0,
 726: 0,
 734: 1,
 751: 1,
 801: 0,
 818: 1,
 862: 1,
 869: 1,
 871: 0,
 894: 0,
 905: 0,
 921: 0,
 928: 1,
 940: 0,
 977: 0,
 1017: 1,
 1054: 0,
 1065: 1,
 1069: 1,
 1070: 1,
 1132: 1,
 1139: 0,
 1219: 1,
 1227: 0,
 1242: 0,
 1249: 1,
 1252: 1,
 1272: 1,
 1275: 1,
 1297: 0,
 1299: 0,
 1301: 0,
 1306: 1,
 1324: 1,
 1371: 0,
 1372: 1,
 1422: 0,
 1438: 1,
 1451: 1,
 1456: 0,
 1518: 1,
 1599: 0,
 1619: 0,
 1656: 0,
 1663: 0,
 1665: 1,
 1675: 1,
 1688: 1,
 1718: 1,
 1727: 0,
 1752: 0,
 1761: 1,
 1816: 1,
 1820: 1,
 1924: 0,
 1925: 1,
 1934: 1,
 1944: 0,
 1979: 0,
 1999: 0,
 2077: 1,
 2144: 0,
 2145: 1,
 2160: 1,
 2

In [33]:
deliverable1 = open('../deliverables/features_svmlight.train', 'wb')
deliverable2 = open('../deliverables/features.train', 'wb')
    
for patient_id in patient_features.keys():
    feature_list = patient_features[patient_id]
    target = mortality_dict[patient_id]
      
    feature_list_string = ",".join(map(
        lambda tuplet: "{feature_id}:{feature_value}".format(
          feature_id=tuplet[0],
          feature_value=tuplet[1]
        ), 
        feature_list
      ))
      
    deliverable1.write(bytes(
        "{target} {feature_value_list}\n".format(
          target=target,
          feature_value_list=feature_list_string
        ), 
        'UTF-8'
    ));
    deliverable2.write(bytes(
        "{patient_id} {label} {feature_value_list}\n".format(
          patient_id=patient_id, 
          label=target,
          feature_value_list=feature_list_string
        ),
        'UTF-8'
    ));